In [57]:
# Load autoreload extension
%load_ext autoreload

# Set autoreload behavior
%autoreload 2
import os
import time
import argparse
import random
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from models.ProTACT import ProTACT
from models.Loss import LossFunctions
from configs.configs import Configs
from utils.read_data_pr import read_pos_vocab, read_word_vocab, read_prompts_we, read_essays_prompts, read_prompts_pos
from utils.general_utils import get_scaled_down_scores, pad_hierarchical_text_sequences, get_attribute_masks, load_word_embedding_dict, build_embedd_table
from evaluators.multitask_evaluator_all_attributes import Evaluator as AllAttEvaluator
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# print("main started")
# parser = argparse.ArgumentParser(description="ProTACT model")
# parser.add_argument('--test_prompt_id', type=int,
#                     default=1, help='prompt id of test essay set')
# parser.add_argument('--seed', type=int, default=12, help='set random seed')
# parser.add_argument('--model_name', type=str,
#                     choices=['ProTACT'],
#                     help='name of model')
# parser.add_argument('--num_heads', type=int, default=2,
#                     help='set the number of heads in Multihead Attention')
# parser.add_argument('--features_path', type=str,
#                     default='data/hand_crafted_v3.csv')
test_prompt_id = 1
seed = 1
num_heads = 2
features_path = '../data/hand_crafted_v3.csv'

np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

print(torch.cuda.is_available())
print("Test prompt id is {} of type {}".format(
    test_prompt_id, type(test_prompt_id)))
print("Seed: {}".format(seed))

configs = Configs()

data_path = configs.DATA_PATH
train_path = data_path + str(test_prompt_id) + '/train.pk'
dev_path = data_path + str(test_prompt_id) + '/dev.pk'
test_path = data_path + str(test_prompt_id) + '/test.pk'
pretrained_embedding = configs.PRETRAINED_EMBEDDING
embedding_path = configs.EMBEDDING_PATH
readability_path = configs.READABILITY_PATH
prompt_path = configs.PROMPT_PATH
vocab_size = configs.VOCAB_SIZE
epochs = configs.EPOCHS
batch_size = configs.BATCH_SIZE
print("Numhead : ", num_heads, " | Features : ",
      features_path, " | Pos_emb : ", configs.EMBEDDING_DIM)

read_configs = {
    'train_path': train_path,
    'dev_path': dev_path,
    'test_path': test_path,
    'features_path': features_path,
    'readability_path': readability_path,
    'vocab_size': vocab_size
}

False
Test prompt id is 1 of type <class 'int'>
Seed: 1
Numhead :  2  | Features :  ../data/hand_crafted_v3.csv  | Pos_emb :  50


In [3]:
# read POS for prompts
pos_vocab = read_pos_vocab(read_configs)
prompt_pos_data = read_prompts_pos(
    prompt_path, pos_vocab)  # for prompt POS embedding

# read words for prompts
word_vocab = read_word_vocab(read_configs)
# for prompt word embedding
prompt_data = read_prompts_we(prompt_path, word_vocab)

train_data, dev_data, test_data = read_essays_prompts(
    read_configs, prompt_data, prompt_pos_data, pos_vocab)

if pretrained_embedding:
    embedd_dict, embedd_dim, _ = load_word_embedding_dict(embedding_path)
    embedd_matrix = build_embedd_table(
        word_vocab, embedd_dict, embedd_dim, caseless=True)
    embed_table = embedd_matrix
else:
    embed_table = None

 prompt_pos size: 8
 prompt_words size: 8
 pos_x size: 9513
 readability_x size: 9513
 pos_x size: 1680
 readability_x size: 1680
 pos_x size: 1783
 readability_x size: 1783
Loading GloVe ...
OOV number =189, OOV ratio = 0.047262


In [4]:
max_sentlen = max(train_data['max_sentlen'],
                  dev_data['max_sentlen'], test_data['max_sentlen'])
max_sentnum = max(train_data['max_sentnum'],
                  dev_data['max_sentnum'], test_data['max_sentnum'])
prompt_max_sentlen = prompt_data['max_sentlen']
prompt_max_sentnum = prompt_data['max_sentnum']

print('max sent length: {}'.format(max_sentlen))
print('max sent num: {}'.format(max_sentnum))
print('max prompt sent length: {}'.format(prompt_max_sentlen))
print('max prompt sent num: {}'.format(prompt_max_sentnum))

train_data['y_scaled'] = get_scaled_down_scores(
    train_data['data_y'], train_data['prompt_ids'])
dev_data['y_scaled'] = get_scaled_down_scores(
    dev_data['data_y'], dev_data['prompt_ids'])
test_data['y_scaled'] = get_scaled_down_scores(
    test_data['data_y'], test_data['prompt_ids'])

X_train_pos = pad_hierarchical_text_sequences(
    train_data['pos_x'], max_sentnum, max_sentlen)
X_dev_pos = pad_hierarchical_text_sequences(
    dev_data['pos_x'], max_sentnum, max_sentlen)
X_test_pos = pad_hierarchical_text_sequences(
    test_data['pos_x'], max_sentnum, max_sentlen)

X_train_pos = X_train_pos.reshape(
    (X_train_pos.shape[0], X_train_pos.shape[1] * X_train_pos.shape[2]))
X_dev_pos = X_dev_pos.reshape(
    (X_dev_pos.shape[0], X_dev_pos.shape[1] * X_dev_pos.shape[2]))
X_test_pos = X_test_pos.reshape(
    (X_test_pos.shape[0], X_test_pos.shape[1] * X_test_pos.shape[2]))

X_train_prompt = pad_hierarchical_text_sequences(
    train_data['prompt_words'], max_sentnum, max_sentlen)
X_dev_prompt = pad_hierarchical_text_sequences(
    dev_data['prompt_words'], max_sentnum, max_sentlen)
X_test_prompt = pad_hierarchical_text_sequences(
    test_data['prompt_words'], max_sentnum, max_sentlen)

X_train_prompt = X_train_prompt.reshape(
    (X_train_prompt.shape[0], X_train_prompt.shape[1] * X_train_prompt.shape[2]))
X_dev_prompt = X_dev_prompt.reshape(
    (X_dev_prompt.shape[0], X_dev_prompt.shape[1] * X_dev_prompt.shape[2]))
X_test_prompt = X_test_prompt.reshape(
    (X_test_prompt.shape[0], X_test_prompt.shape[1] * X_test_prompt.shape[2]))

X_train_prompt_pos = pad_hierarchical_text_sequences(
    train_data['prompt_pos'], max_sentnum, max_sentlen)
X_dev_prompt_pos = pad_hierarchical_text_sequences(
    dev_data['prompt_pos'], max_sentnum, max_sentlen)
X_test_prompt_pos = pad_hierarchical_text_sequences(
    test_data['prompt_pos'], max_sentnum, max_sentlen)

X_train_prompt_pos = X_train_prompt_pos.reshape(
    (X_train_prompt_pos.shape[0], X_train_prompt_pos.shape[1] * X_train_prompt_pos.shape[2]))
X_dev_prompt_pos = X_dev_prompt_pos.reshape(
    (X_dev_prompt_pos.shape[0], X_dev_prompt_pos.shape[1] * X_dev_prompt_pos.shape[2]))
X_test_prompt_pos = X_test_prompt_pos.reshape(
    (X_test_prompt_pos.shape[0], X_test_prompt_pos.shape[1] * X_test_prompt_pos.shape[2]))

X_train_linguistic_features = np.array(train_data['features_x'])
X_dev_linguistic_features = np.array(dev_data['features_x'])
X_test_linguistic_features = np.array(test_data['features_x'])

X_train_readability = np.array(train_data['readability_x'])
X_dev_readability = np.array(dev_data['readability_x'])
X_test_readability = np.array(test_data['readability_x'])

Y_train = np.array(train_data['y_scaled'])
Y_dev = np.array(dev_data['y_scaled'])
Y_test = np.array(test_data['y_scaled'])

X_train_attribute_rel = get_attribute_masks(Y_train)
X_dev_attribute_rel = get_attribute_masks(Y_dev)
X_test_attribute_rel = get_attribute_masks(Y_test)

print('================================')
print('X_train_pos: ', X_train_pos.shape)
print('X_train_prompt_words: ', X_train_prompt.shape)
print('X_train_prompt_pos: ', X_train_prompt_pos.shape)
print('X_train_readability: ', X_train_readability.shape)
print('X_train_ling: ', X_train_linguistic_features.shape)
print('X_train_attribute_rel: ', X_train_attribute_rel.shape)
print('Y_train: ', Y_train.shape)

print('================================')
print('X_dev_pos: ', X_dev_pos.shape)
print('X_dev_prompt_words: ', X_dev_prompt.shape)
print('X_dev_prompt_pos: ', X_dev_prompt_pos.shape)
print('X_dev_readability: ', X_dev_readability.shape)
print('X_dev_ling: ', X_dev_linguistic_features.shape)
print('X_dev_attribute_rel: ', X_dev_attribute_rel.shape)
print('Y_dev: ', Y_dev.shape)

print('================================')
print('X_test_pos: ', X_test_pos.shape)
print('X_test_prompt_words: ', X_test_prompt.shape)
print('X_test_prompt_pos: ', X_test_prompt_pos.shape)
print('X_test_readability: ', X_test_readability.shape)
print('X_test_ling: ', X_test_linguistic_features.shape)
print('X_test_attribute_rel: ', X_test_attribute_rel.shape)
print('Y_test: ', Y_test.shape)
print('================================')

max sent length: 50
max sent num: 97
max prompt sent length: 18
max prompt sent num: 8
X_train_pos:  (9513, 4850)
X_train_prompt_words:  (9513, 4850)
X_train_prompt_pos:  (9513, 4850)
X_train_readability:  (9513, 35)
X_train_ling:  (9513, 51)
X_train_attribute_rel:  (9513, 9)
Y_train:  (9513, 9)
X_dev_pos:  (1680, 4850)
X_dev_prompt_words:  (1680, 4850)
X_dev_prompt_pos:  (1680, 4850)
X_dev_readability:  (1680, 35)
X_dev_ling:  (1680, 51)
X_dev_attribute_rel:  (1680, 9)
Y_dev:  (1680, 9)
X_test_pos:  (1783, 4850)
X_test_prompt_words:  (1783, 4850)
X_test_prompt_pos:  (1783, 4850)
X_test_readability:  (1783, 35)
X_test_ling:  (1783, 51)
X_test_attribute_rel:  (1783, 9)
Y_test:  (1783, 9)


In [5]:
# to torch tensor
train_dataset = TensorDataset(
    torch.from_numpy(X_train_pos),
    torch.from_numpy(X_train_prompt),
    torch.from_numpy(X_train_prompt_pos),
    torch.from_numpy(X_train_linguistic_features),
    torch.from_numpy(X_train_readability),
    torch.from_numpy(Y_train)
)
train_loader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

dev_dataset = TensorDataset(
    torch.from_numpy(X_dev_pos),
    torch.from_numpy(X_dev_prompt),
    torch.from_numpy(X_dev_prompt_pos),
    torch.from_numpy(X_dev_linguistic_features),
    torch.from_numpy(X_dev_readability),
    torch.from_numpy(Y_dev)
)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size)

dev_features_list = [
    torch.from_numpy(X_dev_pos),
    torch.from_numpy(X_dev_prompt),
    torch.from_numpy(X_dev_prompt_pos),
    torch.from_numpy(X_dev_linguistic_features),
    torch.from_numpy(X_dev_readability)
]
test_features_list = [
    torch.from_numpy(X_test_pos),
    torch.from_numpy(X_test_prompt),
    torch.from_numpy(X_test_prompt_pos),
    torch.from_numpy(X_test_linguistic_features),
    torch.from_numpy(X_test_readability)
]

In [58]:
# build model
model = ProTACT(
    len(pos_vocab), len(word_vocab), max_sentnum, max_sentlen,
    X_train_readability.shape[1], X_train_linguistic_features.shape[1],
    configs, Y_train.shape[1], num_heads, embed_table
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# loss function and optimizer
criterion = LossFunctions(alpha=0.7)
optimizer = torch.optim.RMSprop(
    model.parameters(), lr=configs.LEARNING_RATE)

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()}")

self.essay_pos_attention TimeDistributed(
  (module): Attention()
)
Layer: attention_module.att_V | Size: torch.Size([100])
Layer: attention_module.att_W | Size: torch.Size([100, 100])
Layer: essay_pos_embedding.weight | Size: torch.Size([36, 50])
Layer: essay_pos_conv.weight | Size: torch.Size([100, 50, 5])
Layer: essay_pos_conv.bias | Size: torch.Size([100])
Layer: essay_linquistic.weight | Size: torch.Size([100, 51])
Layer: essay_linquistic.bias | Size: torch.Size([100])
Layer: essay_readability.weight | Size: torch.Size([100, 35])
Layer: essay_readability.bias | Size: torch.Size([100])
Layer: essay_pos_MA.0.query_dense.weight | Size: torch.Size([100, 100])
Layer: essay_pos_MA.0.query_dense.bias | Size: torch.Size([100])
Layer: essay_pos_MA.0.key_dense.weight | Size: torch.Size([100, 100])
Layer: essay_pos_MA.0.key_dense.bias | Size: torch.Size([100])
Layer: essay_pos_MA.0.value_dense.weight | Size: torch.Size([100, 100])
Layer: essay_pos_MA.0.value_dense.bias | Size: torch.Size([10

In [59]:
evaluator = AllAttEvaluator(
    test_prompt_id, dev_data['prompt_ids'], test_data['prompt_ids'],
    [x.numpy() for x in dev_features_list],
    [x.numpy() for x in test_features_list], Y_dev, Y_test, seed
)

evaluator.evaluate(model, -1, print_info=True)

pos_drop_x  this is timedistribution checking 4:  torch.Size([1680, 4850, 50])
pos_resh_W torch.Size([1680, 97, 50, 50])
pos_zcnn torch.Size([1680, 97, 100, 46])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (16296000x46 and 100x100)

In [ ]:

class CustomHistory:
    def __init__(self):
        self.train_loss = []
        self.val_loss = []

    def update(self, train_loss, val_loss):
        self.train_loss.append(train_loss)
        self.val_loss.append(val_loss)
        
        
custom_hist = CustomHistory()

for epoch in range(epochs): # 50
    print(f'Epoch {epoch + 1}/{epochs}')
    start_time = time.time()

    # train
    model.train()
    train_loss = 0.0
    for batch_data in train_loader:
        optimizer.zero_grad()
        batch_data = [x.to(device) for x in batch_data]
        inputs, targets = batch_data[:-1], batch_data[-1]
        outputs = model(*inputs)
        loss = criterion.loss_function(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * batch_data[0].size(0)
    train_loss /= len(train_loader.dataset)

    # validate
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch_data in dev_loader:
            batch_data = [x.to(device) for x in batch_data]
            inputs, targets = batch_data[:-1], batch_data[-1]
            outputs = model(*inputs)
            #loss = criterion(outputs, targets)
            loss = criterion.loss_function(outputs, targets)
            val_loss += loss.item() * batch_data[0].size(0)
        val_loss /= len(dev_loader.dataset)

    custom_hist.update(train_loss, val_loss)

    # evaluate
    tt_time = time.time() - start_time
    print(f"Training one epoch in {tt_time:.3f} s")
    evaluator.evaluate(model, epoch + 1)
    print(f"Train Loss: {train_loss:.4f} || Val Loss: {val_loss:.4f}")

evaluator.print_final_info()

'''# show the loss as the graph
fig, loss_graph = plt.subplots()
loss_graph.plot(custom_hist.train_loss,'y',label='train loss')
loss_graph.plot(custom_hist.val_loss,'r',label='val loss')
loss_graph.set_xlabel('epoch')
loss_graph.set_ylabel('loss')
plt.savefig(str('images/protact/test_prompt_'+ str(test_prompt_id) + '_seed_' + str(seed) + '_loss.png'))'''

In [ ]:
Y_train.shape

In [31]:
# TimeDistributed 실험
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, TimeDistributed, Input

# Define input data
batch_size = 32
sentence_num = 7
sentence_length = 5
input_dim = 4
input_data = torch.randn(batch_size,  setence_num,setence_length, input_dim)


# Define input shape
max_sentence_num = 3
max_sentence_length = 5
embedding_dim = 4
input_shape = (setence_num, setence_length, input_dim)

In [26]:
# Define input layer
inputs = Input(shape=input_shape)

# Apply TimeDistributed Conv1D
cnn_filters = 2
cnn_kernel_size = 3
conv1d_output = TimeDistributed(Conv1D(filters=cnn_filters, kernel_size=cnn_kernel_size, padding='valid'))(inputs)

# Define the model
model = tf.keras.Model(inputs=inputs, outputs=conv1d_output)

# Print model summary
model.summary()

# Generate output
output = model.predict(tf.constant(input_data))
print(output.shape)  # Output shape
print(output)  # Output values

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 7, 5, 4)]         0         
                                                                 
 time_distributed_4 (TimeDi  (None, 7, 3, 2)           26        
 stributed)                                                      
                                                                 
Total params: 26 (104.00 Byte)
Trainable params: 26 (104.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
1/1 [==============================] - 0s 28ms/step
(32, 7, 3, 2)
[[[[ 2.22391438e+00  9.05178115e-02]
   [ 1.25336432e+00 -2.58172065e-01]
   [ 1.45193958e+00  7.70615578e-01]]

  [[-9.68962908e-01 -1.24814972e-01]
   [-7.32492566e-01 -1.13072920e+00]
   [ 9.20946479e-01  3.74002874e-01]]

  [[ 6.40787482e-01 -2.10141718e-01]
   [ 1.72482824e+00  3.4

In [37]:
import torch
import torch.nn as nn

input_data = torch.randn(batch_size, sentence_num, sentence_length, input_dim)

# 24 fc timedistributed
num = 24
fc = nn.ModuleList([nn.Conv1d(embedding_dim, cnn_filters, cnn_kernel_size) for i in range(max_sentence_length)])
# forward pass 
outs=[]
for i in range(input_data.shape[1]):
    outs.append(fc[i](input_data[:, i, :].transpose(1,2)))
outs=torch.cat(outs, axis=1)
print(out.shape)
print(out)

IndexError: index 5 is out of range